In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision import datasets
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import copy
from tqdm import tqdm
import optuna
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../Utils')
import configs


In [3]:
class DeepCNN(nn.Module):
    def __init__(self, num_classes=7, dropout_prob=0.5):
        super(DeepCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2,2)
        
        self.conv3 = nn.Conv2d(64,128,3,1,1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128,128,3,1,1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(2,2)
        
        self.conv5 = nn.Conv2d(128,256,3,1,1)
        self.bn5 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(2,2)
        
        self.dropout = nn.Dropout(dropout_prob)
        self.fc1 = nn.Linear(256*28*28, 512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,num_classes)
        
        self._initialize_weights()
        
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
        
    def forward(self,x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool3(x)
        
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


In [4]:
# Training augmentation
transform_train = transforms.Compose([
    transforms.Grayscale(1),
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(0.2),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2,0.2,0.1),
    transforms.RandomAffine(0, translate=(0.1,0.1), scale=(0.9,1.1)),
    transforms.RandomPerspective(0.1,0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Validation / Test (no augmentation)
transform_test = transforms.Compose([
    transforms.Grayscale(1),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Datasets
train_dataset = datasets.ImageFolder(configs.MULTIVIEW_TRAIN_DIR, transform=transform_train)
test_dataset = datasets.ImageFolder(configs.MULTIVIEW_TEST_DIR, transform=transform_test)

# Train / Validation Split
train_size = int(0.8*len(train_dataset))
val_size = len(train_dataset)-train_size
train_subset, val_subset = torch.utils.data.random_split(train_dataset,[train_size,val_size])

val_dataset_no_aug = datasets.ImageFolder(configs.MULTIVIEW_TRAIN_DIR, transform=transform_test)
val_subset_no_aug = Subset(val_dataset_no_aug, val_subset.indices)

# DataLoaders
train_loader = DataLoader(train_subset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_subset_no_aug, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)


In [5]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss, running_corrects, total = 0, 0, 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs,1)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()*inputs.size(0)
        running_corrects += torch.sum(preds==labels.data)
        total += inputs.size(0)
    return running_loss/total, running_corrects.double()/total

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss, running_corrects, total = 0,0,0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs,1)
            loss = criterion(outputs,labels)
            running_loss += loss.item()*inputs.size(0)
            running_corrects += torch.sum(preds==labels.data)
            total += inputs.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return running_loss/total, running_corrects.double()/total, all_labels, all_preds

def train_model_with_history(model, train_loader, val_loader, criterion, optimizer, scheduler=None, 
                             num_epochs=50, patience=10, device=None):
    history = {"train_loss":[],"train_acc":[],"val_loss":[],"val_acc":[]}
    best_acc = 0
    best_model_wts = copy.deepcopy(model.state_dict())
    patience_counter = 0
    if device is None:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc, _, _ = validate(model, val_loader, criterion, device)
        
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc.item())
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc.item())
        
        if scheduler:
            scheduler.step(val_loss)
            
        if val_acc>best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            patience_counter=0
        else:
            patience_counter+=1
            if patience_counter>=patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
        
        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, "
              f"Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model, history


In [6]:
def evaluate_model(model, dataloader, device=None):
    if device is None:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs,1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    acc = accuracy_score(all_labels, all_preds)
    bal_acc = balanced_accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    prec = precision_score(all_labels, all_preds, average='weighted')
    print(f"Accuracy={acc:.4f}, Balanced Acc={bal_acc:.4f}, F1={f1:.4f}, Precision={prec:.4f}")
    print("\nClassification Report:\n", classification_report(all_labels, all_preds))
    return all_labels, all_preds


In [7]:
def plot_learning_curves(history):
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(history["train_loss"], label="Train Loss")
    plt.plot(history["val_loss"], label="Val Loss")
    plt.xlabel("Epochs"); plt.ylabel("Loss"); plt.title("Loss Curve"); plt.legend(); plt.grid(True, alpha=0.3)
    
    plt.subplot(1,2,2)
    plt.plot(history["train_acc"], label="Train Acc")
    plt.plot(history["val_acc"], label="Val Acc")
    plt.xlabel("Epochs"); plt.ylabel("Accuracy"); plt.title("Accuracy Curve"); plt.legend(); plt.grid(True, alpha=0.3)
    
    plt.show()


In [ ]:


# Define search space manually
LR_RANGE = (1e-4, 1e-2)
DROPOUT_RANGE = (0.3, 0.6)
WEIGHT_DECAY_RANGE = (1e-5, 1e-3)

def random_search(n_trials=10):
    best_acc = 0.0
    best_params = {}

    for trial in range(n_trials):
        # Randomly sample hyperparameters
        lr = 10 ** np.random.uniform(np.log10(LR_RANGE[0]), np.log10(LR_RANGE[1]))
        dropout = np.random.uniform(*DROPOUT_RANGE)
        weight_decay = 10 ** np.random.uniform(np.log10(WEIGHT_DECAY_RANGE[0]), np.log10(WEIGHT_DECAY_RANGE[1]))

        print(f"\n🔎 Trial {trial+1}/{n_trials}: lr={lr:.5f}, dropout={dropout:.3f}, weight_decay={weight_decay:.6f}")

        # Subsample training data (30%)
        subset_size = int(0.3 * len(train_subset))
        subset_indices = np.random.choice(train_subset.indices, subset_size, replace=False)
        trial_subset = torch.utils.data.Subset(train_dataset, subset_indices)
        trial_loader = DataLoader(trial_subset, batch_size=16, shuffle=True, num_workers=2)

        # Build model
        model = DeepCNN(num_classes=7, dropout_prob=dropout)
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model.to(device)

        # Compute class weights
        trial_labels = [train_dataset.targets[i] for i in subset_indices]
        classes = np.unique(trial_labels)
        class_weights = compute_class_weight('balanced', classes=classes, y=trial_labels)
        class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

        criterion = nn.CrossEntropyLoss(weight=class_weights)
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)

        # Train for fewer epochs just to test hyperparams quickly
        model, history = train_model_with_history(
            model, trial_loader, val_loader,
            criterion, optimizer, scheduler,
            num_epochs=8,
            device=device,
            patience=3
        )

        val_acc = max(history["val_acc"])
        print(f"✅ Trial {trial+1} validation accuracy: {val_acc:.4f}")

        # Keep track of best
        if val_acc > best_acc:
            best_acc = val_acc
            best_params = {"lr": lr, "dropout": dropout, "weight_decay": weight_decay}

    print("\n🎯 Best Parameters Found:")
    print(best_params)
    print(f"🏆 Best Validation Accuracy: {best_acc:.4f}")
    return best_params, best_acc


# Run random search
best_params, best_acc = random_search(n_trials=10)



🔎 Trial 1/10: lr=0.00601, dropout=0.419, weight_decay=0.000244


In [ ]:
# ===========================
# Final Training with Best Hyperparameters
# ===========================

# Unpack best params from random search
lr = best_params["lr"]
dropout = best_params["dropout"]
weight_decay = best_params["weight_decay"]

print("\n🚀 Training final model with best hyperparameters:")
print(f"lr={lr:.5f}, dropout={dropout:.3f}, weight_decay={weight_decay:.6f}")

# Use full training subset now
full_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=2)

# Build model with best hyperparameters
final_model = DeepCNN(num_classes=7, dropout_prob=dropout)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
final_model.to(device)

# Compute class weights on full training set
train_labels = [train_dataset.targets[i] for i in train_subset.indices]
classes = np.unique(train_labels)
class_weights = compute_class_weight('balanced', classes=classes, y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(final_model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

# Train final model
final_model, final_history = train_model_with_history(
    final_model, full_loader, val_loader,
    criterion, optimizer, scheduler,
    num_epochs=20,  # more epochs for better training now
    device=device,
    patience=5
)

# Best validation accuracy
final_val_acc = max(final_history["val_acc"])
print(f"\n🏆 Final Model Validation Accuracy: {final_val_acc:.4f}")

# Optionally save model
torch.save(final_model.state_dict(), "best_deepcnn_model.pth")
print("💾 Final model saved as best_deepcnn_model.pth")
